In [1]:
# magic reload
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')
from analysis.analysis_utils import *
from analysis.plot_utils import *
from datasets import load_dataset

### Accuracy of QA expert

In [8]:
qa_df = prepare_df(types=['qa'])
filter = {'config_model_name_qa': 'deepseek/deepseek-v3.1-terminus', 'config_dataset_name_qa': 'Idavidrein/gpqa', 'config_dataset_subset_qa': 'gpqa_diamond', 'config_dataset_split_qa': 'train'}
qa_df_deepseek_31_terminus = qa_df[qa_df.apply(lambda row: all(row[col] == val for col, val in filter.items()), axis=1)]

In [9]:
print(qa_df_deepseek_31_terminus['is_correct_qa'].mean().round(2))

0.77


### Analyze Debate

In [10]:
check_df = load_all_records_into_df('debate_checks')
check_df = check_df[check_df['run_id_debate_checks'] == 'z1qae3w']
check_df['has_leak_debate_checks'].value_counts()

has_leak_debate_checks
False    91
True     63
Name: count, dtype: int64

### Analysis Setup

In [11]:
all_df = prepare_df()

# Add the debate leak check
checks_df = load_all_records_into_df('debate_checks').drop_duplicates(['run_id_debate_checks', 'record_id_debate_checks'], keep='last')
all_df = all_df.merge(checks_df, left_on=['debate_run_id_verdicts', 'record_id_verdicts'], right_on=['run_id_debate_checks', 'record_id_debate_checks'], how='left')

,config_debate_run_id_verdicts,config_judge_model_verdicts,config_judge_reasoning_effort_verdicts,config_judge_reasoning_max_tokens_verdicts,config_judge_temperature_verdicts,config_max_output_tokens_verdicts,config_max_threads_verdicts,config_rerun_verdicts,config_skip_qa_verdicts,config_specific_record_ids_verdicts,...,has_leak_debate_checks,leak_types_debate_checks,options_debate_checks,options_str_debate_checks,prompt_template_debate_checks,question_debate_checks,record_id_debate_checks,run_id_debate_checks,success_debate_checks,turns_debate_checks
0,17zguxe,openai/gpt-3.5-turbo,None,None,0.0,5000,NaN,NaN,NaN,NaN,...,True,{'1': 'Explicitly stating or implying that the...,"[2,3,4, 1,2,4]","['2,3,4', '1,2,4']",You are analyzing a debate argument to detect ...,A common approximation made in many-body nucle...,6ekpf86,17zguxe,True,"[{'debater_idx': 0, 'turn_idx': 0, 'public_arg..."
1,17zguxe,openai/gpt-3.5-turbo,None,None,0.0,5000,NaN,NaN,NaN,NaN,...,True,{'1': 'Explicitly stating or implying that the...,[The eigenfunction of the operator Ay can also...,['The eigenfunction of the operator Ay can als...,You are analyzing a debate argument to detect ...,Consider the Y-component of the intrinsic angu...,fas8lyj,17zguxe,True,"[{'debater_idx': 1, 'turn_idx': 0, 'public_arg..."
2,17zguxe,openai/gpt-3.5-turbo,None,None,0.0,5000,NaN,NaN,NaN,NaN,...,False,{'1': 'Explicitly stating or implying that the...,"[1.9, 3]","['1.9', '3']",You are analyzing a debate argument to detect ...,Large clouds of neutral hydrogen in deep space...,gmbiyoa,17zguxe,True,"[{'debater_idx': 1, 'turn_idx': 0, 'public_arg..."
3,17zguxe,openai/gpt-3.5-turbo,None,None,0.0,5000,NaN,NaN,NaN,NaN,...,False,{'1': 'Explicitly stating or implying that the...,"[ribosome to the proteasome, cytosol to the ex...","['ribosome to the proteasome', 'cytosol to the...",You are analyzing a debate argument to detect ...,"""Oh, I know you,"" the ribonucleoprotein partic...",7opss8e,17zguxe,True,"[{'debater_idx': 0, 'turn_idx': 0, 'public_arg..."
4,17zguxe,openai/gpt-3.5-turbo,None,None,0.0,5000,NaN,NaN,NaN,NaN,...,False,{'1': 'Explicitly stating or implying that the...,"[All of the above, 3 and 4]","['All of the above', '3 and 4']",You are analyzing a debate argument to detect ...,Which of the following issues are the most com...,7fuq5qd,17zguxe,True,"[{'debater_idx': 0, 'turn_idx': 0, 'public_arg..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14926,q6wpwb7,qwen/qwen3-235b-a22b,None,None,0.0,5000,5000.0,NaN,0.0,None,...,True,{'1': 'Explicitly stating or implying that the...,"[3.52 MeV, 20.8 MeV, 2.84 MeV, 26.8 MeV, 4.12 ...","['3.52 MeV, 20.8 MeV', '2.84 MeV, 26.8 MeV', '...",You are analyzing a debate argument to detect ...,"Find KE of product particles in,\nPi(+) = mu(+...",mkbw0d2,q6wpwb7,True,"[{'debater_idx': 0, 'turn_idx': 0, 'public_arg..."
14927,q6wpwb7,qwen/qwen3-235b-a22b,None,None,0.0,5000,5000.0,NaN,0.0,None,...,False,{'1': 'Explicitly stating or implying that the...,"[1. Zn, ether\n2. Cl2/hv\n3. Aq. KOH\n4. Pyrid...","['1. Zn, ether\n2. Cl2/hv\n3. Aq. KOH\n4. Pyri...",You are analyzing a debate argument to detect ...,Identify the correct sequence of reagents for ...,e27r9c0,q6wpwb7,True,"[{'debater_idx': 1, 'turn_idx': 0, 'public_arg..."
14928,q6wpwb7,qwen/qwen3-235b-a22b,None,None,0.0,5000,5000.0,NaN,0.0,None,...,False,{'1': 'Explicitly stating or implying that the...,"[35, 25, 31, 29]","['35', '25', '31', '29']",You are analyzing a debate argument to detect ...,"Substance X, known for incorporating a heavier...",to8a7h8,q6wpwb7,True,"[{'debater_idx': 1, 'turn_idx': 0, 'public_arg..."
14929,q6wpwb7,qwen/qwen3-235b-a22b,None,None,0.0,5000,5000.0,NaN,0.0,None,...,False,{'1': 'Explicitly stating or implying that the...,"[v_rel = 1.1c , E= mc^2, v_rel = 0.14c , E= 5....","['v_rel = 1.1c , E= mc^2', 'v_rel = 0.14c , E=...",You are analyzing a debate argument to detect ...,Two astronauts of masses 2m and 3m travel at r...,99mo404,